In [2]:
#%pip install torch==2.5.1 torchvision==0.20.1 torchaudio==2.5.1 --index-url https://download.pytorch.org/whl/cu124 --force-reinstall

In [3]:
"""import torch

def format_pytorch_version(version_str):
    # Example input: "2.0.1+cu118" -> returns "2.0.1"
    return version_str.split('+')[0]

def format_cuda_version(cuda_str):
    # If CUDA is None (CPU-only PyTorch), return "cpu"
    if cuda_str is None:
        return "cpu"
    # Example: "11.8" -> "cu118"
    return "cu" + cuda_str.replace('.', '')

TORCH_version = torch.__version__
TORCH = format_pytorch_version(TORCH_version)
CUDA_version = torch.version.cuda
CUDA = format_cuda_version(CUDA_version)

print(f"Installing packages for torch-{TORCH}+{CUDA}...")

%pip install --upgrade --no-cache-dir pyg_lib torch_scatter torch_sparse torch_cluster torch_spline_conv -f https://data.pyg.org/whl/torch-{TORCH}+{CUDA}.html

%pip install torch_geometric"""

'import torch\n\ndef format_pytorch_version(version_str):\n    # Example input: "2.0.1+cu118" -> returns "2.0.1"\n    return version_str.split(\'+\')[0]\n\ndef format_cuda_version(cuda_str):\n    # If CUDA is None (CPU-only PyTorch), return "cpu"\n    if cuda_str is None:\n        return "cpu"\n    # Example: "11.8" -> "cu118"\n    return "cu" + cuda_str.replace(\'.\', \'\')\n\nTORCH_version = torch.__version__\nTORCH = format_pytorch_version(TORCH_version)\nCUDA_version = torch.version.cuda\nCUDA = format_cuda_version(CUDA_version)\n\nprint(f"Installing packages for torch-{TORCH}+{CUDA}...")\n\n%pip install --upgrade --no-cache-dir pyg_lib torch_scatter torch_sparse torch_cluster torch_spline_conv -f https://data.pyg.org/whl/torch-{TORCH}+{CUDA}.html\n\n%pip install torch_geometric'

In [4]:
#%pip install scikit-learn imbalanced-learn xgboost

In [5]:
#%pip install optuna

### Hyperparameter Optimization Citeseer
#### SCAR 

In [13]:
from data_generating import make_pu_dataset,load_dataset
data=load_dataset("citeseer")
data=make_pu_dataset(data)

# Convert the sum of train_mask to a Python int
labeled = data.train_mask.sum().item()  

# If you need the total number of nodes (rows in data.x):
total = data.x.size(0)  # or data.x.shape[0]
ratio=(0.286026*(total-labeled)-0.5/0.35*labeled)/(total-labeled -0.5/0.35*labeled)
print(ratio)

0.18671026440138042


In [1]:
from train_NNIF_GNN import run_nnif_gnn_experiment
import optuna
from typing import Dict, Any

def objective(trial: optuna.trial.Trial) -> float:
    """
    Objective function for hyperparameter optimization using Optuna.
    It builds a parameter dictionary, calls the experiment function, and
    returns the average F1 score.
    """
    params: Dict[str, Any] = {
        "dataset_name": "citeseer",      
        "mechanism": "SCAR",
        "train_pct": 0.5,
        "K": trial.suggest_int("K", 2, 22),
        "layers": trial.suggest_int("layers", 1, 3),
        "hidden_channels": trial.suggest_categorical("hidden_channels", [64,128, 256]),
        "out_channels": trial.suggest_categorical("out_channels", [64,128, 256]),
        "norm": None,
        "dropout": 0,
        "ratio": trial.suggest_float("ratio", 0.12, 0.3),
        "aggregation": 'mean',
        "treatment": 'removal',#trial.suggest_categorical("treatment", ["removal", "relabeling"]),
        "model_type": trial.suggest_categorical("model_type", ['GCNConv', 'GATConv']),
        "rate_pairs": 10,#trial.suggest_int("rate_pairs", 1, 15),
        "batch_size": trial.suggest_categorical("batch_size", [1024,2048]),
        "lr":0.005,# trial.suggest_float("lr", 1e-4, 1e-2),
        "anomaly_detector":"nearest_neighbors",#trial.suggest_categorical("anomaly_detector",["nearest_neighbors","unweighted"]),
        "seeds": 5,
        "output_csv": "citeseer_test.csv",
        "min":0.71,
        "clusters":50,#trial.suggest_categorical("clusters", [50,100,200,300,400]),          
        "num_epochs":100,         
        "sampling":"sage",#trial.suggest_categorical("sampling",["sage"]),#,"cluster"])#,"shine"]),
        "val":False    ,
        "methodology":"ours"     
        
    }
    
    avg_f1, std_f1 = run_nnif_gnn_experiment(params)
    
    # We aim to maximize F1 score.
    return avg_f1

# Create an Optuna study to maximize the F1 score.
study = optuna.create_study(direction="maximize")
study.optimize(objective, n_trials=100)

# Print out the best hyperparameters and corresponding F1 score.
print("Best trial:")
trial = study.best_trial
print("  Average F1:", trial.value)
print("  Best parameters:")
for key, value in trial.params.items():
    print(f"    {key}: {value}")


[I 2025-04-15 21:41:39,006] A new study created in memory with name: no-name-98ccc7a5-b286-4869-af35-d59df7617277


Running experiment with seed=654:
 - K=14, layers=2, hidden=64, out=128
 - norm=None, dropout=0, batch_size=1024, methodology=ours
 - ratio=0.27213409059615434, aggregation=mean, treatment=removal, anomaly_detector=nearest_neighbors, sampling=sage
 - model_type=GATConv, rate_pairs=10, clusters=50, lr=0.005
here
Epoch 0 / 100, Loss: 5.7060
Epoch 10 / 100, Loss: 5.2189
Epoch 20 / 100, Loss: 5.0410
Epoch 30 / 100, Loss: 4.7343
Epoch 40 / 100, Loss: 4.1818
Epoch 50 / 100, Loss: 3.7451
Epoch 60 / 100, Loss: 3.5046
Epoch 70 / 100, Loss: 3.2639
Epoch 80 / 100, Loss: 2.9313
Epoch 90 / 100, Loss: 2.5293


[I 2025-04-15 21:42:30,685] Trial 0 finished with value: 0.5841726618705037 and parameters: {'K': 14, 'layers': 2, 'hidden_channels': 64, 'out_channels': 128, 'ratio': 0.27213409059615434, 'model_type': 'GATConv', 'batch_size': 1024}. Best is trial 0 with value: 0.5841726618705037.


 - Test Metrics: Accuracy=0.9029, F1=0.5842, Recall=0.5783, Precision=0.5901
 - Validation Metrics: Accuracy=nan, F1=0.0000, Recall=0.0000, Precision=0.0000
F1 = 0.00 < 0.71, skipping ...
Done. Results written to citeseer_experimentations\citeseer_test_1504214139.csv.
Average F1 over valid seeds: 0.5842 ± 0.0000
Running experiment with seed=654:
 - K=4, layers=1, hidden=64, out=64
 - norm=None, dropout=0, batch_size=2048, methodology=ours
 - ratio=0.18283894777588056, aggregation=mean, treatment=removal, anomaly_detector=nearest_neighbors, sampling=sage
 - model_type=GCNConv, rate_pairs=10, clusters=50, lr=0.005
here
Epoch 0 / 100, Loss: 2.1797
Epoch 10 / 100, Loss: 1.4945
Epoch 20 / 100, Loss: 1.4249
Epoch 30 / 100, Loss: 1.3591
Epoch 40 / 100, Loss: 1.2905
Epoch 50 / 100, Loss: 1.2165
Epoch 60 / 100, Loss: 1.1555
Epoch 70 / 100, Loss: 1.0925
Epoch 80 / 100, Loss: 1.0080
Epoch 90 / 100, Loss: 0.9447


[I 2025-04-15 21:42:46,405] Trial 1 finished with value: 0.6203554119547657 and parameters: {'K': 4, 'layers': 1, 'hidden_channels': 64, 'out_channels': 64, 'ratio': 0.18283894777588056, 'model_type': 'GCNConv', 'batch_size': 2048}. Best is trial 1 with value: 0.6203554119547657.


...

 - Test Metrics: Accuracy=0.9308, F1=0.6988, Recall=0.6809, Precision=0.7177
 - Validation Metrics: Accuracy=nan, F1=0.0000, Recall=0.0000, Precision=0.0000
F1 = 0.00 < 0.71, skipping ...
Done. Results written to citeseer_experimentations\citeseer_test_1504231124.csv.
Average F1 over valid seeds: 0.7119 ± 0.0085
Running experiment with seed=654:
 - K=4, layers=1, hidden=64, out=64
 - norm=None, dropout=0, batch_size=2048, methodology=ours
 - ratio=0.2763236181335535, aggregation=mean, treatment=removal, anomaly_detector=nearest_neighbors, sampling=sage
 - model_type=GCNConv, rate_pairs=10, clusters=50, lr=0.005
here
Epoch 0 / 100, Loss: 2.1600
Epoch 10 / 100, Loss: 1.5020
Epoch 20 / 100, Loss: 1.4353
Epoch 30 / 100, Loss: 1.3700
Epoch 40 / 100, Loss: 1.2993
Epoch 50 / 100, Loss: 1.2247
Epoch 60 / 100, Loss: 1.1607
Epoch 70 / 100, Loss: 1.0982
Epoch 80 / 100, Loss: 1.0144
Epoch 90 / 100, Loss: 0.9529


[I 2025-04-15 23:13:10,476] Trial 98 finished with value: 0.6531204644412192 and parameters: {'K': 4, 'layers': 1, 'hidden_channels': 64, 'out_channels': 64, 'ratio': 0.2763236181335535, 'model_type': 'GCNConv', 'batch_size': 2048}. Best is trial 58 with value: 0.728373464830273.


 - Test Metrics: Accuracy=0.9197, F1=0.6531, Recall=0.6410, Precision=0.6657
 - Validation Metrics: Accuracy=nan, F1=0.0000, Recall=0.0000, Precision=0.0000
F1 = 0.00 < 0.71, skipping ...
Done. Results written to citeseer_experimentations\citeseer_test_1504231254.csv.
Average F1 over valid seeds: 0.6531 ± 0.0000
Running experiment with seed=654:
 - K=2, layers=1, hidden=64, out=256
 - norm=None, dropout=0, batch_size=2048, methodology=ours
 - ratio=0.14726399082556924, aggregation=mean, treatment=removal, anomaly_detector=nearest_neighbors, sampling=sage
 - model_type=GCNConv, rate_pairs=10, clusters=50, lr=0.005
here
Epoch 0 / 100, Loss: 1.7636
Epoch 10 / 100, Loss: 1.0012
Epoch 20 / 100, Loss: 0.9411
Epoch 30 / 100, Loss: 0.8611
Epoch 40 / 100, Loss: 0.8191
Epoch 50 / 100, Loss: 0.7449
Epoch 60 / 100, Loss: 0.7074
Epoch 70 / 100, Loss: 0.6738
Epoch 80 / 100, Loss: 0.6154
Epoch 90 / 100, Loss: 0.5745


[I 2025-04-15 23:13:32,721] Trial 99 finished with value: 0.6555740432612313 and parameters: {'K': 2, 'layers': 1, 'hidden_channels': 64, 'out_channels': 256, 'ratio': 0.14726399082556924, 'model_type': 'GCNConv', 'batch_size': 2048}. Best is trial 58 with value: 0.728373464830273.


 - Test Metrics: Accuracy=0.9305, F1=0.6556, Recall=0.5613, Precision=0.7880
 - Validation Metrics: Accuracy=nan, F1=0.0000, Recall=0.0000, Precision=0.0000
F1 = 0.00 < 0.71, skipping ...
Done. Results written to citeseer_experimentations\citeseer_test_1504231310.csv.
Average F1 over valid seeds: 0.6556 ± 0.0000
Best trial:
  Average F1: 0.728373464830273
  Best parameters:
    K: 9
    layers: 1
    hidden_channels: 128
    out_channels: 256
    ratio: 0.2671400383263691
    model_type: GCNConv
    batch_size: 2048


### Hyperparameter Optimization Citeseer
#### SAR 

In [4]:
from train_NNIF_GNN import run_nnif_gnn_experiment
import optuna
from typing import Dict, Any

def objective(trial: optuna.trial.Trial) -> float:
    """
    Objective function for hyperparameter optimization using Optuna.
    It builds a parameter dictionary, calls the experiment function, and
    returns the average F1 score.
    """
    params: Dict[str, Any] = {
        "dataset_name": "citeseer",      
        "mechanism": "SAR",
        "train_pct": 0.5,
        "K": trial.suggest_int("K", 2, 20),
        "layers": trial.suggest_int("layers", 1, 3),
        "hidden_channels": trial.suggest_categorical("hidden_channels", [64,128, 256]),
        "out_channels": trial.suggest_categorical("out_channels", [64,128, 256]),
        "norm": None,
        "dropout": 0,
        "ratio": trial.suggest_float("ratio", 0.09, 0.18),
        "aggregation": 'mean',
        "treatment": "removal",#trial.suggest_categorical("treatment", ["removal", "relabeling"]),
        "model_type": trial.suggest_categorical("model_type", ['GCNConv', 'GATConv']),
        "rate_pairs":trial.suggest_int("layers", 1, 15),
        "batch_size": trial.suggest_categorical("batch_size", [256,512,1024]),
        "lr": trial.suggest_float("lr", 1e-4, 1e-2),
        "anomaly_detector":trial.suggest_categorical("anomaly_detector",["nearest_neighbors","unweighted"]),
        "seeds": 5,
        "output_csv": "citeseer_test.csv",
        "min":0.745,
        "clusters":trial.suggest_categorical("clusters", [50,100,200,300,400]),
        "num_epochs":100,         
        "sampling":trial.suggest_categorical("sampling",["sage","cluster","shine"]),         
        
       
    }
    
    avg_f1, std_f1 = run_nnif_gnn_experiment(params)
    
    # We aim to maximize F1 score.
    return avg_f1

# Create an Optuna study to maximize the F1 score.
study = optuna.create_study(direction="maximize")
study.optimize(objective, n_trials=200)

# Print out the best hyperparameters and corresponding F1 score.
print("Best trial:")
trial = study.best_trial
print("  Average F1:", trial.value)
print("  Best parameters:")
for key, value in trial.params.items():
    print(f"    {key}: {value}")


[I 2025-04-15 21:11:08,636] A new study created in memory with name: no-name-65948645-0843-48b3-acdd-5c02de781cc9
[W 2025-04-15 21:11:08,642] Trial 0 failed with parameters: {'K': 12, 'layers': 1, 'hidden_channels': 128, 'out_channels': 128, 'ratio': 0.0937795709369779, 'model_type': 'GATConv', 'batch_size': 1024, 'lr': 0.005074524462670685, 'anomaly_detector': 'unweighted', 'clusters': 200, 'sampling': 'shine'} because of the following error: KeyError('methodology').
Traceback (most recent call last):
  File "c:\Users\romai\Desktop\gnn\gnn_pu\.conda\Lib\site-packages\optuna\study\_optimize.py", line 197, in _run_trial
    value_or_values = func(trial)
                      ^^^^^^^^^^^
  File "C:\Users\romai\AppData\Local\Temp\ipykernel_19744\168235557.py", line 39, in objective
    avg_f1, std_f1 = run_nnif_gnn_experiment(params)
                     ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\romai\Desktop\gnn\gnn_pu\NNIF-GNN\train_NNIF_GNN.py", line 467, in run_nnif_gnn_experim

KeyError: 'methodology'

### Hyperparameter Optimization Cora
#### SCAR 

In [5]:
from data_generating import make_pu_dataset,load_dataset
data=load_dataset("cora")
data=make_pu_dataset(data)

# Convert the sum of train_mask to a Python int
labeled = data.train_mask.sum().item()  

# If you need the total number of nodes (rows in data.x):
total = data.x.size(0)  # or data.x.shape[0]
ratio=(0.379923*(total-labeled)-0.5/0.35*labeled)/(total-labeled -0.5/0.35*labeled)
print(ratio)

0.24134569855135768


In [2]:
from train_NNIF_GNN import run_nnif_gnn_experiment
import optuna
from typing import Dict, Any

def objective(trial: optuna.trial.Trial) -> float:
    """
    Objective function for hyperparameter optimization using Optuna.
    It builds a parameter dictionary, calls the experiment function, and
    returns the average F1 score.
    """
    params: Dict[str, Any] = {
        "dataset_name": "cora",      
        "mechanism": "SCAR",
        "train_pct": 0.5,
        "K": trial.suggest_int("K", 1, 25),
        "layers": trial.suggest_int("layers", 1, 3),
        "hidden_channels": trial.suggest_categorical("hidden_channels", [64,128, 256]),
        "out_channels": trial.suggest_categorical("out_channels", [64,128, 256]),
        "norm": None,
        "dropout": 0,
        "ratio": trial.suggest_float("ratio", 0.16, 0.38),
        "aggregation": "mean",#trial.suggest_categorical("aggregation", ['mean', 'sum','max']),
        "treatment": "removal",#trial.suggest_categorical("treatment", ["removal", "relabeling"]),
        "model_type": trial.suggest_categorical("model_type", ['GCNConv', 'GATConv']),
        "rate_pairs":10,#trial.suggest_int("rate_pairs", 1, 15),
        "batch_size": trial.suggest_categorical("batch_size", [1024,2048]),
        "lr":0.005,# trial.suggest_float("lr", 1e-4, 1e-2),
        "anomaly_detector":"nearest_neighbors",#trial.suggest_categorical("anomaly_detector",["nearest_neighbors","unweighted"]),
        "seeds": 5,
        "output_csv": "cora_test.csv",
        "min":0.795,
        "clusters":50,#trial.suggest_categorical("clusters", [50,100,200,300,400]),          
        "num_epochs":100,         
        "sampling":trial.suggest_categorical("sampling",["sage"]),#,"cluster"])#,"shine"]),     
        "val":False,    
        "methodology":"ours"
        
    }
    
    avg_f1, std_f1 = run_nnif_gnn_experiment(params)
    
    # We aim to maximize F1 score.
    return avg_f1

# Create an Optuna study to maximize the F1 score.
study = optuna.create_study(direction="maximize")
study.optimize(objective, n_trials=100)

# Print out the best hyperparameters and corresponding F1 score.
print("Best trial:")
trial = study.best_trial
print("  Average F1:", trial.value)
print("  Best parameters:")
for key, value in trial.params.items():
    print(f"    {key}: {value}")


[I 2025-04-15 23:13:32,759] A new study created in memory with name: no-name-741195ac-81fc-4f39-9854-a4032b37c4e5


Running experiment with seed=654:
 - K=22, layers=3, hidden=128, out=256
 - norm=None, dropout=0, batch_size=1024, methodology=ours
 - ratio=0.28702279110394624, aggregation=mean, treatment=removal, anomaly_detector=nearest_neighbors, sampling=sage
 - model_type=GCNConv, rate_pairs=10, clusters=50, lr=0.005
here
Epoch 0 / 100, Loss: 4.6885
Epoch 10 / 100, Loss: 4.2693
Epoch 20 / 100, Loss: 4.1053
Epoch 30 / 100, Loss: 4.0523
Epoch 40 / 100, Loss: 3.8401
Epoch 50 / 100, Loss: 3.7994
Epoch 60 / 100, Loss: 3.6436
Epoch 70 / 100, Loss: 3.4725
Epoch 80 / 100, Loss: 3.3702
Epoch 90 / 100, Loss: 3.1252


[I 2025-04-15 23:14:02,072] Trial 0 finished with value: 0.7941534713763703 and parameters: {'K': 22, 'layers': 3, 'hidden_channels': 128, 'out_channels': 256, 'ratio': 0.28702279110394624, 'model_type': 'GCNConv', 'batch_size': 1024, 'sampling': 'sage'}. Best is trial 0 with value: 0.7941534713763703.


 - Test Metrics: Accuracy=0.9265, F1=0.7942, Recall=0.7971, Precision=0.7913
 - Validation Metrics: Accuracy=nan, F1=0.0000, Recall=0.0000, Precision=0.0000
F1 = 0.00 < 0.795, skipping ...
Done. Results written to cora_experimentations\cora_test_1504231332.csv.
Average F1 over valid seeds: 0.7942 ± 0.0000
Running experiment with seed=654:
 - K=20, layers=3, hidden=128, out=128
 - norm=None, dropout=0, batch_size=2048, methodology=ours
 - ratio=0.27664307329107574, aggregation=mean, treatment=removal, anomaly_detector=nearest_neighbors, sampling=sage
 - model_type=GCNConv, rate_pairs=10, clusters=50, lr=0.005
here
Epoch 0 / 100, Loss: 3.0122
Epoch 10 / 100, Loss: 2.9817
Epoch 20 / 100, Loss: 2.7641
Epoch 30 / 100, Loss: 2.5677
Epoch 40 / 100, Loss: 2.7930
Epoch 50 / 100, Loss: 2.6558
Epoch 60 / 100, Loss: 2.5720
Epoch 70 / 100, Loss: 2.4565
Epoch 80 / 100, Loss: 2.3879
Epoch 90 / 100, Loss: 2.2922


[I 2025-04-15 23:14:21,546] Trial 1 finished with value: 0.7712895377128953 and parameters: {'K': 20, 'layers': 3, 'hidden_channels': 128, 'out_channels': 128, 'ratio': 0.27664307329107574, 'model_type': 'GCNConv', 'batch_size': 2048, 'sampling': 'sage'}. Best is trial 0 with value: 0.7941534713763703.


...

 - Test Metrics: Accuracy=0.9178, F1=0.7855, Recall=0.8460, Precision=0.7331
 - Validation Metrics: Accuracy=nan, F1=0.0000, Recall=0.0000, Precision=0.0000
F1 = 0.00 < 0.795, skipping ...
Done. Results written to cora_experimentations\cora_test_1604001855.csv.
Average F1 over valid seeds: 0.7855 ± 0.0000
Running experiment with seed=654:
 - K=23, layers=3, hidden=128, out=256
 - norm=None, dropout=0, batch_size=1024, methodology=ours
 - ratio=0.2778461271562757, aggregation=mean, treatment=removal, anomaly_detector=nearest_neighbors, sampling=sage
 - model_type=GCNConv, rate_pairs=10, clusters=50, lr=0.005
here
Epoch 0 / 100, Loss: 4.7535
Epoch 10 / 100, Loss: 4.3231
Epoch 20 / 100, Loss: 4.1515
Epoch 30 / 100, Loss: 4.1047
Epoch 40 / 100, Loss: 3.8969
Epoch 50 / 100, Loss: 3.8556
Epoch 60 / 100, Loss: 3.6891
Epoch 70 / 100, Loss: 3.5385
Epoch 80 / 100, Loss: 3.4433
Epoch 90 / 100, Loss: 3.2061


[I 2025-04-16 00:19:44,723] Trial 98 finished with value: 0.7884615384615384 and parameters: {'K': 23, 'layers': 3, 'hidden_channels': 128, 'out_channels': 256, 'ratio': 0.2778461271562757, 'model_type': 'GCNConv', 'batch_size': 1024, 'sampling': 'sage'}. Best is trial 94 with value: 0.806246419779795.


 - Test Metrics: Accuracy=0.9234, F1=0.7885, Recall=0.8020, Precision=0.7754
 - Validation Metrics: Accuracy=nan, F1=0.0000, Recall=0.0000, Precision=0.0000
F1 = 0.00 < 0.795, skipping ...
Done. Results written to cora_experimentations\cora_test_1604001916.csv.
Average F1 over valid seeds: 0.7885 ± 0.0000
Running experiment with seed=654:
 - K=16, layers=3, hidden=256, out=256
 - norm=None, dropout=0, batch_size=1024, methodology=ours
 - ratio=0.3150743629825921, aggregation=mean, treatment=removal, anomaly_detector=nearest_neighbors, sampling=sage
 - model_type=GCNConv, rate_pairs=10, clusters=50, lr=0.005
here
Epoch 0 / 100, Loss: 4.1079
Epoch 10 / 100, Loss: 4.0187
Epoch 20 / 100, Loss: 3.8050
Epoch 30 / 100, Loss: 3.5814
Epoch 40 / 100, Loss: 3.5572
Epoch 50 / 100, Loss: 3.4282
Epoch 60 / 100, Loss: 3.2226
Epoch 70 / 100, Loss: 3.0719
Epoch 80 / 100, Loss: 2.8166
Epoch 90 / 100, Loss: 2.5680
 - Test Metrics: Accuracy=0.9334, F1=0.8211, Recall=0.8582, Precision=0.7870
 - Validation 

[I 2025-04-16 00:20:39,749] Trial 99 finished with value: 0.8007702182284981 and parameters: {'K': 16, 'layers': 3, 'hidden_channels': 256, 'out_channels': 256, 'ratio': 0.3150743629825921, 'model_type': 'GCNConv', 'batch_size': 1024, 'sampling': 'sage'}. Best is trial 94 with value: 0.806246419779795.


 - Test Metrics: Accuracy=0.9217, F1=0.7805, Recall=0.7824, Precision=0.7786
 - Validation Metrics: Accuracy=nan, F1=0.0000, Recall=0.0000, Precision=0.0000
F1 = 0.00 < 0.795, skipping ...
Done. Results written to cora_experimentations\cora_test_1604001944.csv.
Average F1 over valid seeds: 0.8008 ± 0.0203
Best trial:
  Average F1: 0.806246419779795
  Best parameters:
    K: 23
    layers: 3
    hidden_channels: 256
    out_channels: 256
    ratio: 0.3076334769040583
    model_type: GCNConv
    batch_size: 1024
    sampling: sage


### Hyperparameter Optimization Cora
#### SAR 

In [ ]:
from train_NNIF_GNN import run_nnif_gnn_experiment
import optuna
from typing import Dict, Any

def objective(trial: optuna.trial.Trial) -> float:
    """
    Objective function for hyperparameter optimization using Optuna.
    It builds a parameter dictionary, calls the experiment function, and
    returns the average F1 score.
    """
    params: Dict[str, Any] = {
        "dataset_name": "cora",      
        "mechanism": "SAR",
        "train_pct": 0.5,
        "K": trial.suggest_int("K", 7, 9),
        "layers": trial.suggest_int("layers", 1, 2),
        "hidden_channels": trial.suggest_categorical("hidden_channels", [64,128, 256]),
        "out_channels": trial.suggest_categorical("out_channels", [64,128, 256]),
        "norm": None,
        "dropout": 0,
        "ratio": trial.suggest_float("ratio", 0.16, 0.34),
        "aggregation": 'mean',
        "treatment": "removal",#trial.suggest_categorical("treatment", ["removal", "relabeling"]),
        "model_type": trial.suggest_categorical("model_type", ['GCNConv', 'GATConv']),
        "rate_pairs":trial.suggest_int("rate_pairs", 1, 15),
        "batch_size": trial.suggest_categorical("batch_size", [5,10,15]),
        "lr": trial.suggest_float("lr", 1e-4, 1e-2),
        "anomaly_detector":trial.suggest_categorical("anomaly_detector",["nearest_neighbors","unweighted"]),
        "seeds": 5,
        "output_csv": "cora_sar_sampling_nnif_batch_cluster.csv",
        "min":0.85,
        "clusters":trial.suggest_categorical("clusters", [50,100,200,300,400]),          
        "num_epochs":trial.suggest_categorical("num_epochs", [50,100,150]),         
        "sampling":"cluster",#trial.suggest_categorical("sampling",["sage","cluster","shine"]),         
        
       
    }
    
    avg_f1, std_f1 = run_nnif_gnn_experiment(params)
    
    # We aim to maximize F1 score.
    return avg_f1

# Create an Optuna study to maximize the F1 score.
study = optuna.create_study(direction="maximize")
study.optimize(objective, n_trials=100)

# Print out the best hyperparameters and corresponding F1 score.
print("Best trial:")
trial = study.best_trial
print("  Average F1:", trial.value)
print("  Best parameters:")
for key, value in trial.params.items():
    print(f"    {key}: {value}")


[I 2025-03-26 15:51:16,342] A new study created in memory with name: no-name-8176da62-abb1-46e4-8807-5e5ff1459ce0


Running experiment with seed=654:
 - K=9, layers=1, hidden=64, out=256
 - norm=None, dropout=0, batch_size=1024, methodology=ours
 - ratio=0.16889523261113468, aggregation=mean, treatment=removal, anomaly_detector=nearest_neighbors, sampling=neighbor, sampling_k=75
 - model_type=GCNConv, rate_pairs=1, clusters=400, lr=0.007646163484926425
Epoch 0 / 150, Loss: 3.4303
Epoch 10 / 150, Loss: 3.1501
Epoch 20 / 150, Loss: 2.8808
Epoch 30 / 150, Loss: 2.6330
Epoch 40 / 150, Loss: 2.3545
Epoch 50 / 150, Loss: 2.0481
Epoch 60 / 150, Loss: 1.7211
Epoch 70 / 150, Loss: 1.4545
Epoch 80 / 150, Loss: 1.2300
Epoch 90 / 150, Loss: 1.0373
Epoch 100 / 150, Loss: 0.8788
Epoch 110 / 150, Loss: 0.7530
Epoch 120 / 150, Loss: 0.6579
Epoch 130 / 150, Loss: 0.5790
Epoch 140 / 150, Loss: 0.5190
 - Metrics: Accuracy=0.9228, F1=0.8656, Recall=0.8227, Precision=0.9132
Running experiment with seed=114:
 - K=9, layers=1, hidden=64, out=256
 - norm=None, dropout=0, batch_size=1024, methodology=ours
 - ratio=0.1688952

[I 2025-03-26 15:53:50,635] Trial 0 finished with value: 0.8684716037566916 and parameters: {'K': 9, 'layers': 1, 'hidden_channels': 64, 'out_channels': 256, 'ratio': 0.16889523261113468, 'model_type': 'GCNConv', 'batch_size': 1024, 'lr': 0.007646163484926425, 'anomaly_detector': 'nearest_neighbors', 'clusters': 400, 'num_epochs': 150, 'sampling': 'neighbor', 'sampling_k': 75}. Best is trial 0 with value: 0.8684716037566916.


 - Metrics: Accuracy=0.9269, F1=0.8731, Recall=0.8325, Precision=0.9178
Done. Results written to cora_experimentations\cora_sar_sampling_nnif_batch_cluster_2603155116.csv.
Average F1 over valid seeds: 0.8685 ± 0.0049
Running experiment with seed=654:
 - K=9, layers=1, hidden=128, out=256
 - norm=None, dropout=0, batch_size=512, methodology=ours
 - ratio=0.1886114414198443, aggregation=mean, treatment=removal, anomaly_detector=unweighted, sampling=nearest_neighbors, sampling_k=25
 - model_type=GCNConv, rate_pairs=1, clusters=200, lr=0.00016096248213070527
Epoch 0 / 150, Loss: 35.3035
Epoch 10 / 150, Loss: 35.6685
Epoch 20 / 150, Loss: 34.8467
Epoch 30 / 150, Loss: 34.7338
Epoch 40 / 150, Loss: 34.8928
Epoch 50 / 150, Loss: 34.1160
Epoch 60 / 150, Loss: 33.9046
Epoch 70 / 150, Loss: 33.9726
Epoch 80 / 150, Loss: 33.1283
Epoch 90 / 150, Loss: 33.2420
Epoch 100 / 150, Loss: 32.6670
Epoch 110 / 150, Loss: 32.7155
[Early Stopping] at epoch 118


[I 2025-03-26 15:54:30,723] Trial 1 finished with value: 0.6825775656324582 and parameters: {'K': 9, 'layers': 1, 'hidden_channels': 128, 'out_channels': 256, 'ratio': 0.1886114414198443, 'model_type': 'GCNConv', 'batch_size': 512, 'lr': 0.00016096248213070527, 'anomaly_detector': 'unweighted', 'clusters': 200, 'num_epochs': 150, 'sampling': 'nearest_neighbors', 'sampling_k': 25}. Best is trial 0 with value: 0.8684716037566916.


...

Epoch 0 / 100, Loss: 5.9769
Epoch 10 / 100, Loss: 5.1580
Epoch 20 / 100, Loss: 4.5073
Epoch 30 / 100, Loss: 3.6713
Epoch 40 / 100, Loss: 3.1895
Epoch 50 / 100, Loss: 2.3100
Epoch 60 / 100, Loss: 1.8741
Epoch 70 / 100, Loss: 1.5376
Epoch 80 / 100, Loss: 1.2265
Epoch 90 / 100, Loss: 1.0520
 - Metrics: Accuracy=0.9210, F1=0.8694, Recall=0.8704, Precision=0.8683
Running experiment with seed=25:
 - K=8, layers=1, hidden=128, out=256
 - norm=None, dropout=0, batch_size=1024, methodology=ours
 - ratio=0.2573033125681119, aggregation=mean, treatment=removal, anomaly_detector=nearest_neighbors, sampling=cluster, sampling_k=25
 - model_type=GCNConv, rate_pairs=1, clusters=100, lr=0.007397697366787508


Computing METIS partitioning...
Done!


Epoch 0 / 100, Loss: 5.8094


[W 2025-03-26 16:49:22,637] Trial 31 failed with parameters: {'K': 8, 'layers': 1, 'hidden_channels': 128, 'out_channels': 256, 'ratio': 0.2573033125681119, 'model_type': 'GCNConv', 'batch_size': 1024, 'lr': 0.007397697366787508, 'anomaly_detector': 'nearest_neighbors', 'clusters': 100, 'num_epochs': 100, 'sampling': 'cluster', 'sampling_k': 25} because of the following error: KeyboardInterrupt().
Traceback (most recent call last):
  File "c:\Users\romai\Desktop\gnn\gnn_pu\.conda\Lib\site-packages\optuna\study\_optimize.py", line 197, in _run_trial
    value_or_values = func(trial)
                      ^^^^^^^^^^^
  File "C:\Users\romai\AppData\Local\Temp\ipykernel_39988\45902056.py", line 39, in objective
    avg_f1, std_f1 = run_nnif_gnn_experiment(params)
                     ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\romai\Desktop\gnn\gnn_pu\NNIF-GNN\train_NNIF_GNN.py", line 565, in run_nnif_gnn_experiment
    train_labels, train_proba, train_losses = train_graph(
          

KeyboardInterrupt: 

### Hyperparameter Optimization Pubmed
#### SCAR 

In [8]:
from data_generating import make_pu_dataset,load_dataset
data=load_dataset("pubmed")
data=make_pu_dataset(data)

# Convert the sum of train_mask to a Python int
labeled = data.train_mask.sum().item()  

# If you need the total number of nodes (rows in data.x):
total = data.x.size(0)  # or data.x.shape[0]
ratio=(0.489441*(total-labeled)-0.5/0.35*labeled)/(total-labeled -0.5/0.35*labeled)
print(ratio)

0.31766718352866136


In [3]:
from train_NNIF_GNN import run_nnif_gnn_experiment
import optuna
from typing import Dict, Any

def objective(trial: optuna.trial.Trial) -> float:
    """
    Objective function for hyperparameter optimization using Optuna.
    It builds a parameter dictionary, calls the experiment function, and
    returns the average F1 score.
    """
    params: Dict[str, Any] = {
        "dataset_name": "pubmed",      
        "mechanism": "SAR",
        "train_pct": 0.5,
        "K": trial.suggest_int("K", 2, 30),
        "layers": trial.suggest_int("layers", 1, 3),
        "hidden_channels": trial.suggest_categorical("hidden_channels", [64,128, 256]),
        "out_channels": trial.suggest_categorical("out_channels", [64,128, 256]),
        "norm": None,
        "dropout": 0,
        "ratio":trial.suggest_float("ratio", 0.2, 0.5),
        "aggregation": 'mean',
        "treatment": "removal",#trial.suggest_categorical("treatment", ["removal", "relabeling"]),
        "model_type": trial.suggest_categorical("model_type", ['GCNConv', 'GATConv']),
        "rate_pairs":10,#trial.suggest_int("rate_pairs", 1, 15),
        "batch_size": trial.suggest_categorical("batch_size", [1024,2048,4096]),
        "lr":0.005,# trial.suggest_float("lr", 1e-4, 1e-2),
        "anomaly_detector": "nearest_neighbors",#trial.suggest_categorical("anomaly_detector",["nearest_neighbors","unweighted"]),
        "seeds": 5,
        "output_csv": "pubmed_scr_val.csv",
        "min":0.775,
        "clusters":50,#trial.suggest_categorical("clusters", [50,100,200,300,400]),          
        "num_epochs":100,         
        "sampling":"sage",#trial.suggest_categorical("sampling",["sage"]),#,"cluster"])#,"shine"]),     
        "val":False,
        "methodology":"ours"    
    }
    
    avg_f1, std_f1 = run_nnif_gnn_experiment(params)
    
    # We aim to maximize F1 score.
    return avg_f1

# Create an Optuna study to maximize the F1 score.
study = optuna.create_study(direction="maximize")
study.optimize(objective, n_trials=100)

# Print out the best hyperparameters and corresponding F1 score.
print("Best trial:")
trial = study.best_trial
print("  Average F1:", trial.value)
print("  Best parameters:")
for key, value in trial.params.items():
    print(f"    {key}: {value}")


[I 2025-04-16 00:20:39,780] A new study created in memory with name: no-name-b33896ed-8e67-4a04-92a3-514141ca018a


Running experiment with seed=654:
 - K=24, layers=1, hidden=64, out=64
 - norm=None, dropout=0, batch_size=2048, methodology=ours
 - ratio=0.3248714188413634, aggregation=mean, treatment=removal, anomaly_detector=nearest_neighbors, sampling=sage
 - model_type=GATConv, rate_pairs=10, clusters=50, lr=0.005
here
Epoch 0 / 100, Loss: 22.1411
Epoch 10 / 100, Loss: 18.7609
Epoch 20 / 100, Loss: 16.2779
Epoch 30 / 100, Loss: 13.6074
Epoch 40 / 100, Loss: 10.3794
Epoch 50 / 100, Loss: 7.4088
Epoch 60 / 100, Loss: 5.2311
Epoch 70 / 100, Loss: 3.8338
Epoch 80 / 100, Loss: 2.9162
Epoch 90 / 100, Loss: 2.3086


[I 2025-04-16 00:22:43,566] Trial 0 finished with value: 0.5523861766319254 and parameters: {'K': 24, 'layers': 1, 'hidden_channels': 64, 'out_channels': 64, 'ratio': 0.3248714188413634, 'model_type': 'GATConv', 'batch_size': 2048}. Best is trial 0 with value: 0.5523861766319254.


 - Test Metrics: Accuracy=0.7931, F1=0.5524, Recall=0.5116, Precision=0.6003
 - Validation Metrics: Accuracy=nan, F1=0.0000, Recall=0.0000, Precision=0.0000
F1 = 0.00 < 0.775, skipping ...
Done. Results written to pubmed_experimentations\pubmed_scr_val_1604002039.csv.
Average F1 over valid seeds: 0.5524 ± 0.0000
Running experiment with seed=654:
 - K=17, layers=1, hidden=128, out=64
 - norm=None, dropout=0, batch_size=4096, methodology=ours
 - ratio=0.355320111183831, aggregation=mean, treatment=removal, anomaly_detector=nearest_neighbors, sampling=sage
 - model_type=GATConv, rate_pairs=10, clusters=50, lr=0.005
here
Epoch 0 / 100, Loss: 10.5029
Epoch 10 / 100, Loss: 9.3345
Epoch 20 / 100, Loss: 8.6119
Epoch 30 / 100, Loss: 7.8580
Epoch 40 / 100, Loss: 7.1145
Epoch 50 / 100, Loss: 6.3357
Epoch 60 / 100, Loss: 5.5156
Epoch 70 / 100, Loss: 4.7345
Epoch 80 / 100, Loss: 4.0174
Epoch 90 / 100, Loss: 3.3848


[I 2025-04-16 00:23:50,596] Trial 1 finished with value: 0.6268849078489496 and parameters: {'K': 17, 'layers': 1, 'hidden_channels': 128, 'out_channels': 64, 'ratio': 0.355320111183831, 'model_type': 'GATConv', 'batch_size': 4096}. Best is trial 1 with value: 0.6268849078489496.


...

 - Test Metrics: Accuracy=0.8559, F1=0.7292, Recall=0.7772, Precision=0.6867
 - Validation Metrics: Accuracy=nan, F1=0.0000, Recall=0.0000, Precision=0.0000
F1 = 0.00 < 0.775, skipping ...
Done. Results written to pubmed_experimentations\pubmed_scr_val_1604061156.csv.
Average F1 over valid seeds: 0.7292 ± 0.0000
Running experiment with seed=654:
 - K=6, layers=2, hidden=128, out=256
 - norm=None, dropout=0, batch_size=4096, methodology=ours
 - ratio=0.3658498966853724, aggregation=mean, treatment=removal, anomaly_detector=nearest_neighbors, sampling=sage
 - model_type=GATConv, rate_pairs=10, clusters=50, lr=0.005
here
Epoch 0 / 100, Loss: 6.3479
Epoch 10 / 100, Loss: 5.6056
Epoch 20 / 100, Loss: 4.9029
Epoch 30 / 100, Loss: 4.2222
Epoch 40 / 100, Loss: 3.5858
Epoch 50 / 100, Loss: 3.0105
Epoch 60 / 100, Loss: 2.5578
Epoch 70 / 100, Loss: 2.1683
Epoch 80 / 100, Loss: 1.8382
Epoch 90 / 100, Loss: 1.5849


[I 2025-04-16 06:15:34,843] Trial 98 finished with value: 0.7291145274699082 and parameters: {'K': 6, 'layers': 2, 'hidden_channels': 128, 'out_channels': 256, 'ratio': 0.3658498966853724, 'model_type': 'GATConv', 'batch_size': 4096}. Best is trial 73 with value: 0.7304768486523843.


 - Test Metrics: Accuracy=0.8559, F1=0.7291, Recall=0.7770, Precision=0.6868
 - Validation Metrics: Accuracy=nan, F1=0.0000, Recall=0.0000, Precision=0.0000
F1 = 0.00 < 0.775, skipping ...
Done. Results written to pubmed_experimentations\pubmed_scr_val_1604061345.csv.
Average F1 over valid seeds: 0.7291 ± 0.0000
Running experiment with seed=654:
 - K=6, layers=2, hidden=128, out=256
 - norm=None, dropout=0, batch_size=4096, methodology=ours
 - ratio=0.36896756565025873, aggregation=mean, treatment=removal, anomaly_detector=nearest_neighbors, sampling=sage
 - model_type=GATConv, rate_pairs=10, clusters=50, lr=0.005
here
Epoch 0 / 100, Loss: 6.3492
Epoch 10 / 100, Loss: 5.6080
Epoch 20 / 100, Loss: 4.9068
Epoch 30 / 100, Loss: 4.2226
Epoch 40 / 100, Loss: 3.5883
Epoch 50 / 100, Loss: 3.0128
Epoch 60 / 100, Loss: 2.5591
Epoch 70 / 100, Loss: 2.1693
Epoch 80 / 100, Loss: 1.8403
Epoch 90 / 100, Loss: 1.5858


[I 2025-04-16 06:17:24,561] Trial 99 finished with value: 0.7282453637660485 and parameters: {'K': 6, 'layers': 2, 'hidden_channels': 128, 'out_channels': 256, 'ratio': 0.36896756565025873, 'model_type': 'GATConv', 'batch_size': 4096}. Best is trial 73 with value: 0.7304768486523843.


 - Test Metrics: Accuracy=0.8551, F1=0.7282, Recall=0.7780, Precision=0.6845
 - Validation Metrics: Accuracy=nan, F1=0.0000, Recall=0.0000, Precision=0.0000
F1 = 0.00 < 0.775, skipping ...
Done. Results written to pubmed_experimentations\pubmed_scr_val_1604061534.csv.
Average F1 over valid seeds: 0.7282 ± 0.0000
Best trial:
  Average F1: 0.7304768486523843
  Best parameters:
    K: 6
    layers: 2
    hidden_channels: 128
    out_channels: 256
    ratio: 0.40961245106780436
    model_type: GATConv
    batch_size: 4096


### Hyperparameter Optimization Pubmed
#### SAR 

In [ ]:
from train_NNIF_GNN import run_nnif_gnn_experiment
import optuna
from typing import Dict, Any

def objective(trial: optuna.trial.Trial) -> float:
    """
    Objective function for hyperparameter optimization using Optuna.
    It builds a parameter dictionary, calls the experiment function, and
    returns the average F1 score.
    """
    params: Dict[str, Any] = {
        "dataset_name": "pubmed",      
        "mechanism": "SAR",
        "train_pct": 0.5,
        "K": trial.suggest_int("K", 2, 12),
        "layers": trial.suggest_int("layers", 1, 2),
        "hidden_channels": trial.suggest_categorical("hidden_channels", [64,128, 256]),
        "out_channels": trial.suggest_categorical("out_channels", [64,128, 256]),
        "norm": None,
        "dropout": 0,
        "ratio": trial.suggest_float("ratio", 0.2, 0.4),
        "aggregation": 'mean',
        "treatment": "removal",#trial.suggest_categorical("treatment", ["removal", "relabeling"]),
        "model_type": trial.suggest_categorical("model_type", ['GCNConv', 'GATConv']),
        "rate_pairs":trial.suggest_int("layers", 1, 15),
        "batch_size": trial.suggest_categorical("batch_size", [15,256,512,1024]),
        "lr": trial.suggest_float("lr", 1e-4, 1e-2),
        "anomaly_detector":"nearest_neighbors",#trial.suggest_categorical("anomaly_detector",["nearest_neighbors","unweighted"]),
        "seeds": 5,
        "output_csv": "pubmed_sar_sampling_nnif_batch_cluster.csv",
        "min":0.84,
        "clusters":trial.suggest_categorical("clusters", [50,100,200,300,400]),          
        "num_epochs":trial.suggest_categorical("num_epochs", [50,100,150]),         
        "sampling":trial.suggest_categorical("sampling",["sage","cluster","shine"]),         
        
       
    }
    
    avg_f1, std_f1 = run_nnif_gnn_experiment(params)
    
    # We aim to maximize F1 score.
    return avg_f1

# Create an Optuna study to maximize the F1 score.
study = optuna.create_study(direction="maximize")
study.optimize(objective, n_trials=200)

# Print out the best hyperparameters and corresponding F1 score.
print("Best trial:")
trial = study.best_trial
print("  Average F1:", trial.value)
print("  Best parameters:")
for key, value in trial.params.items():
    print(f"    {key}: {value}")


[I 2025-03-24 05:51:18,349] A new study created in memory with name: no-name-7363eccd-5b06-40dc-87b0-25014df8c5a1


Running experiment with seed=654:
 - K=7, layers=2, hidden=128, out=256
 - norm=None, dropout=0, batch_size=1024, methodology=ours
 - ratio=0.3362908893186948, aggregation=mean, treatment=removal, anomaly_detector=unweighted, sampling=cluster, sampling_k=50
 - model_type=GATConv, rate_pairs=10, clusters=300, lr=0.0007087214622711031


Computing METIS partitioning...
Done!


Epoch 0 / 100, Loss: 18.5522
Epoch 10 / 100, Loss: 17.8089
Epoch 20 / 100, Loss: 17.3418
Epoch 30 / 100, Loss: 16.8934
Epoch 40 / 100, Loss: 16.1184
Epoch 50 / 100, Loss: 15.1700
Epoch 60 / 100, Loss: 14.4180
Epoch 70 / 100, Loss: 13.4842
Epoch 80 / 100, Loss: 12.6036
Epoch 90 / 100, Loss: 11.9384


[I 2025-03-24 05:53:00,820] Trial 0 finished with value: 0.715743932941256 and parameters: {'K': 7, 'ratio': 0.3362908893186948, 'model_type': 'GATConv', 'batch_size': 1024, 'lr': 0.0007087214622711031, 'anomaly_detector': 'unweighted', 'clusters': 300, 'num_epochs': 100, 'sampling': 'cluster', 'sampling_k': 50}. Best is trial 0 with value: 0.715743932941256.


...

 - Metrics: Accuracy=0.8745, F1=0.8391, Recall=0.8193, Precision=0.8598
F1 = 0.84 < 0.84, skipping ...
Done. Results written to pubmed_experimentations\pubmed_sar_sampling_nnif_batch_cluster_2403133828.csv.
Average F1 over valid seeds: 0.8391 ± 0.0000
Running experiment with seed=654:
 - K=7, layers=2, hidden=128, out=256
 - norm=None, dropout=0, batch_size=512, methodology=ours
 - ratio=0.28001597838526016, aggregation=mean, treatment=removal, anomaly_detector=nearest_neighbors, sampling=nearest_neighbors, sampling_k=50
 - model_type=GCNConv, rate_pairs=10, clusters=400, lr=0.00873922908266354
Epoch 0 / 50, Loss: 58.0806
Epoch 10 / 50, Loss: 8.0599
Epoch 20 / 50, Loss: 2.8407
Epoch 30 / 50, Loss: 1.4709
Epoch 40 / 50, Loss: 0.8989
 - Metrics: Accuracy=0.8762, F1=0.8413, Recall=0.8212, Precision=0.8624
Running experiment with seed=114:
 - K=7, layers=2, hidden=128, out=256
 - norm=None, dropout=0, batch_size=512, methodology=ours
 - ratio=0.28001597838526016, aggregation=mean, treatmen

[I 2025-03-24 15:01:39,121] Trial 13 finished with value: 0.835498509343777 and parameters: {'K': 7, 'ratio': 0.28001597838526016, 'model_type': 'GCNConv', 'batch_size': 512, 'lr': 0.00873922908266354, 'anomaly_detector': 'nearest_neighbors', 'clusters': 400, 'num_epochs': 50, 'sampling': 'nearest_neighbors', 'sampling_k': 50}. Best is trial 10 with value: 0.8395798210348853.


 - Metrics: Accuracy=0.8702, F1=0.8297, Recall=0.7916, Precision=0.8716
F1 = 0.83 < 0.84, skipping ...
Done. Results written to pubmed_experimentations\pubmed_sar_sampling_nnif_batch_cluster_2403141459.csv.
Average F1 over valid seeds: 0.8355 ± 0.0058
Running experiment with seed=654:
 - K=7, layers=2, hidden=128, out=256
 - norm=None, dropout=0, batch_size=512, methodology=ours
 - ratio=0.26308150498044675, aggregation=mean, treatment=removal, anomaly_detector=nearest_neighbors, sampling=nearest_neighbors, sampling_k=50
 - model_type=GCNConv, rate_pairs=10, clusters=400, lr=0.009817622280543212
Epoch 0 / 50, Loss: 57.2382
Epoch 10 / 50, Loss: 6.7150
Epoch 20 / 50, Loss: 2.3497


[W 2025-03-24 15:11:12,619] Trial 14 failed with parameters: {'K': 7, 'ratio': 0.26308150498044675, 'model_type': 'GCNConv', 'batch_size': 512, 'lr': 0.009817622280543212, 'anomaly_detector': 'nearest_neighbors', 'clusters': 400, 'num_epochs': 50, 'sampling': 'nearest_neighbors', 'sampling_k': 50} because of the following error: KeyboardInterrupt().
Traceback (most recent call last):
  File "c:\Users\romai\Desktop\gnn\gnn_pu\.conda\Lib\site-packages\optuna\study\_optimize.py", line 197, in _run_trial
    value_or_values = func(trial)
                      ^^^^^^^^^^^
  File "C:\Users\romai\AppData\Local\Temp\ipykernel_22248\4249684348.py", line 39, in objective
    avg_f1, std_f1 = run_nnif_gnn_experiment(params)
                     ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\romai\Desktop\gnn\gnn_pu\NNIF-GNN\train_NNIF_GNN.py", line 565, in run_nnif_gnn_experiment
    train_labels, train_proba, train_losses = train_graph(
                                              ^^^^^^^^^^^

KeyboardInterrupt: 

### Hyperparameter Optimization WikiCS
#### SCAR 

In [12]:
from data_generating import make_pu_dataset,load_dataset
data=load_dataset("wiki-cs")
data=make_pu_dataset(data)

# Convert the sum of train_mask to a Python int
labeled = data.train_mask.sum().item()  

# If you need the total number of nodes (rows in data.x):
total = data.x.size(0)  # or data.x.shape[0]
ratio=(0.291999*(total-labeled)-0.5/0.35*labeled)/(total-labeled -0.5/0.35*labeled)
print(ratio)

0.1822305564160316


In [ ]:
from train_NNIF_GNN import run_nnif_gnn_experiment
import optuna
from typing import Dict, Any

def objective(trial: optuna.trial.Trial) -> float:
    """
    Objective function for hyperparameter optimization using Optuna.
    It builds a parameter dictionary, calls the experiment function, and
    returns the average F1 score.
    """
    params: Dict[str, Any] = {
        "dataset_name": "wiki-cs",      
        "mechanism": "SCAR",
        "train_pct": 0.5,
        "K": trial.suggest_int("K", 2, 25),
        "layers": trial.suggest_int("layers", 1, 3),
        "hidden_channels": trial.suggest_categorical("hidden_channels", [64,128, 256]),
        "out_channels": trial.suggest_categorical("out_channels", [64,128, 256]),
        "norm": None,
        "dropout": 0,
        "ratio": trial.suggest_float("ratio", 0.12, 0.35),
        "aggregation": 'mean',
        "treatment": "removal",#trial.suggest_categorical("treatment", ["removal", "relabeling"]),
        "model_type": trial.suggest_categorical("model_type", ['GCNConv', 'GATConv']),
        "rate_pairs":10,#trial.suggest_int("rate_pairs",1,15),
        "batch_size": trial.suggest_categorical("batch_size", [1024,2048,4096]),
        "lr": 0.005,
        "anomaly_detector":"nearest_neighbors",#trial.suggest_categorical("anomaly_detector",["nearest_neighbors","unweighted"]),
        "seeds": 5,
        "output_csv": "wikics_scar_val.csv",
        "min":0.815,
        "clusters":50,#trial.suggest_categorical("clusters", [50,100,200,300,400]),          
        "num_epochs":100,      
        "sampling":"sage",#trial.suggest_categorical("sampling",["sage"])#,"cluster"])#,"shine"]),         
        "val":False,
        "methodology":"ours"
       
    }
    
    avg_f1, std_f1 = run_nnif_gnn_experiment(params)
    
    # We aim to maximize F1 score.
    return avg_f1

# Create an Optuna study to maximize the F1 score.
study = optuna.create_study(direction="maximize")
study.optimize(objective, n_trials=100)

# Print out the best hyperparameters and corresponding F1 score.
print("Best trial:")
trial = study.best_trial
print("  Average F1:", trial.value)
print("  Best parameters:")
for key, value in trial.params.items():
    print(f"    {key}: {value}")


[I 2025-04-16 08:39:03,171] A new study created in memory with name: no-name-6530d731-206b-4b9d-abec-902cac0ec006


Running experiment with seed=654:
 - K=8, layers=3, hidden=256, out=64
 - norm=None, dropout=0, batch_size=1024, methodology=ours
 - ratio=0.3239976888643746, aggregation=mean, treatment=removal, anomaly_detector=nearest_neighbors, sampling=sage
 - model_type=GCNConv, rate_pairs=10, clusters=50, lr=0.005
here
Epoch 0 / 100, Loss: 15.8941
Epoch 10 / 100, Loss: 12.1579
Epoch 20 / 100, Loss: 8.2537
Epoch 30 / 100, Loss: 5.3735
Epoch 40 / 100, Loss: 3.6884
Epoch 50 / 100, Loss: 2.6327
Epoch 60 / 100, Loss: 1.9904
Epoch 70 / 100, Loss: 1.5673
Epoch 80 / 100, Loss: 1.2629
Epoch 90 / 100, Loss: 1.0398


[I 2025-04-16 08:41:41,191] Trial 0 finished with value: 0.7339280221470317 and parameters: {'K': 8, 'layers': 3, 'hidden_channels': 256, 'out_channels': 64, 'ratio': 0.3239976888643746, 'model_type': 'GCNConv', 'batch_size': 1024}. Best is trial 0 with value: 0.7339280221470317.


 - Test Metrics: Accuracy=0.9165, F1=0.7339, Recall=0.8910, Precision=0.6240
 - Validation Metrics: Accuracy=nan, F1=0.0000, Recall=0.0000, Precision=0.0000
F1 = 0.00 < 0.815, skipping ...
Done. Results written to wiki-cs_experimentations\wikics_scar_val_1604083903.csv.
Average F1 over valid seeds: 0.7339 ± 0.0000
Running experiment with seed=654:
 - K=14, layers=3, hidden=128, out=128
 - norm=None, dropout=0, batch_size=4096, methodology=ours
 - ratio=0.22668839753269998, aggregation=mean, treatment=removal, anomaly_detector=nearest_neighbors, sampling=sage
 - model_type=GATConv, rate_pairs=10, clusters=50, lr=0.005
here
Epoch 0 / 100, Loss: 5.4073
Epoch 10 / 100, Loss: 5.2230
Epoch 20 / 100, Loss: 5.0049
Epoch 30 / 100, Loss: 4.7034
Epoch 40 / 100, Loss: 4.4276
Epoch 50 / 100, Loss: 4.1545
Epoch 60 / 100, Loss: 3.8775
Epoch 70 / 100, Loss: 3.6222
Epoch 80 / 100, Loss: 3.3360
Epoch 90 / 100, Loss: 3.0943


[I 2025-04-16 08:42:34,505] Trial 1 finished with value: 0.5464247598719317 and parameters: {'K': 14, 'layers': 3, 'hidden_channels': 128, 'out_channels': 128, 'ratio': 0.22668839753269998, 'model_type': 'GATConv', 'batch_size': 4096}. Best is trial 0 with value: 0.7339280221470317.


...

 - Test Metrics: Accuracy=0.8738, F1=0.2205, Recall=0.1382, Precision=0.5457
 - Validation Metrics: Accuracy=nan, F1=0.0000, Recall=0.0000, Precision=0.0000
F1 = 0.00 < 0.815, skipping ...
Done. Results written to wiki-cs_experimentations\wikics_scar_val_1604085804.csv.
Average F1 over valid seeds: 0.2205 ± 0.0000
Running experiment with seed=654:
 - K=25, layers=3, hidden=256, out=128
 - norm=None, dropout=0, batch_size=1024, methodology=ours
 - ratio=0.14987980308164262, aggregation=mean, treatment=removal, anomaly_detector=nearest_neighbors, sampling=sage
 - model_type=GCNConv, rate_pairs=10, clusters=50, lr=0.005
here
Epoch 0 / 100, Loss: 23.5190
Epoch 10 / 100, Loss: 19.7659
Epoch 20 / 100, Loss: 16.9540
Epoch 30 / 100, Loss: 13.1874
Epoch 40 / 100, Loss: 8.7987
Epoch 50 / 100, Loss: 5.5880
Epoch 60 / 100, Loss: 3.8412
Epoch 70 / 100, Loss: 2.8001
Epoch 80 / 100, Loss: 2.1390
Epoch 90 / 100, Loss: 1.6906
 - Test Metrics: Accuracy=0.9566, F1=0.8242, Recall=0.7879, Precision=0.8640


[I 2025-04-16 09:05:47,341] Trial 11 finished with value: 0.8148871527777778 and parameters: {'K': 25, 'layers': 3, 'hidden_channels': 256, 'out_channels': 128, 'ratio': 0.14987980308164262, 'model_type': 'GCNConv', 'batch_size': 1024}. Best is trial 11 with value: 0.8148871527777778.


 - Test Metrics: Accuracy=0.9514, F1=0.8056, Recall=0.7797, Precision=0.8332
 - Validation Metrics: Accuracy=nan, F1=0.0000, Recall=0.0000, Precision=0.0000
F1 = 0.00 < 0.815, skipping ...
Done. Results written to wiki-cs_experimentations\wikics_scar_val_1604090004.csv.
Average F1 over valid seeds: 0.8149 ± 0.0093
Running experiment with seed=654:
 - K=25, layers=3, hidden=256, out=128
 - norm=None, dropout=0, batch_size=1024, methodology=ours
 - ratio=0.12492832119770297, aggregation=mean, treatment=removal, anomaly_detector=nearest_neighbors, sampling=sage
 - model_type=GCNConv, rate_pairs=10, clusters=50, lr=0.005
here
Epoch 0 / 100, Loss: 23.6795
Epoch 10 / 100, Loss: 19.8295
Epoch 20 / 100, Loss: 17.0144
Epoch 30 / 100, Loss: 13.2656
Epoch 40 / 100, Loss: 8.8827
Epoch 50 / 100, Loss: 5.6496
Epoch 60 / 100, Loss: 3.8821
Epoch 70 / 100, Loss: 2.8280
Epoch 80 / 100, Loss: 2.1585
Epoch 90 / 100, Loss: 1.7057


[I 2025-04-16 09:08:33,267] Trial 12 finished with value: 0.8037922506183017 and parameters: {'K': 25, 'layers': 3, 'hidden_channels': 256, 'out_channels': 128, 'ratio': 0.12492832119770297, 'model_type': 'GCNConv', 'batch_size': 1024}. Best is trial 11 with value: 0.8148871527777778.


 - Test Metrics: Accuracy=0.9541, F1=0.8038, Recall=0.7282, Precision=0.8970
 - Validation Metrics: Accuracy=nan, F1=0.0000, Recall=0.0000, Precision=0.0000
F1 = 0.00 < 0.815, skipping ...
Done. Results written to wiki-cs_experimentations\wikics_scar_val_1604090547.csv.
Average F1 over valid seeds: 0.8038 ± 0.0000
Running experiment with seed=654:
 - K=25, layers=1, hidden=256, out=128
 - norm=None, dropout=0, batch_size=1024, methodology=ours
 - ratio=0.17435674991603128, aggregation=mean, treatment=removal, anomaly_detector=nearest_neighbors, sampling=sage
 - model_type=GCNConv, rate_pairs=10, clusters=50, lr=0.005
here
Epoch 0 / 100, Loss: 29.7067
Epoch 10 / 100, Loss: 20.3691
Epoch 20 / 100, Loss: 17.0615
Epoch 30 / 100, Loss: 13.2685
Epoch 40 / 100, Loss: 8.9790


### Hyperparameter Optimization WikiCS
#### SAR 

In [14]:
from train_NNIF_GNN import run_nnif_gnn_experiment
import optuna
from typing import Dict, Any

def objective(trial: optuna.trial.Trial) -> float:
    """
    Objective function for hyperparameter optimization using Optuna.
    It builds a parameter dictionary, calls the experiment function, and
    returns the average F1 score.
    """
    params: Dict[str, Any] = {
        "dataset_name": "wiki-cs",      
        "mechanism": "SAR",
        "train_pct": 0.5,
        "K": 3,#trial.suggest_int("K", 2, 35),
        "layers": 2,#trial.suggest_int("layers", 1, 2),
        "hidden_channels": 256,#trial.suggest_categorical("hidden_channels", [64,128, 256]),
        "out_channels": 64,#trial.suggest_categorical("out_channels", [64,128, 256]),
        "norm": None,
        "dropout": 0,
        "ratio": 0.1822305564160316,#trial.suggest_float("ratio", 0.15, 0.35),
        "aggregation": 'mean',
        "treatment": "removal",#trial.suggest_categorical("treatment", ["removal", "relabeling"]),
        "model_type": "GCNConv",#trial.suggest_categorical("model_type", ['GCNConv', 'GATConv']),
        "rate_pairs":10,#trial.suggest_int("rate_pairs",1,15),
        "batch_size": 2048,#trial.suggest_categorical("batch_size", [1024,2048]),
        "lr": 0.005,
        "anomaly_detector":"nearest_neighbors",#trial.suggest_categorical("anomaly_detector",["nearest_neighbors","unweighted"]),
        "seeds": 5,
        "output_csv": "wikics_sar_val.csv",
        "min":0.85,
        "clusters":50,#trial.suggest_categorical("clusters", [50,100,200,300,400]),          
        "num_epochs":100,      
        "sampling":"sage",#trial.suggest_categorical("sampling",["sage"])#,"cluster"])#,"shine"]),         
        "val":False,
       
    }
    
    avg_f1, std_f1 = run_nnif_gnn_experiment(params)
    
    # We aim to maximize F1 score.
    return avg_f1

# Create an Optuna study to maximize the F1 score.
study = optuna.create_study(direction="maximize")
study.optimize(objective, n_trials=1)

# Print out the best hyperparameters and corresponding F1 score.
print("Best trial:")
trial = study.best_trial
print("  Average F1:", trial.value)
print("  Best parameters:")
for key, value in trial.params.items():
    print(f"    {key}: {value}")


[I 2025-04-14 22:00:29,047] A new study created in memory with name: no-name-1a4f9b7d-322b-4c27-bab1-ba7a9f3fd599


Running experiment with seed=654:
 - K=3, layers=2, hidden=256, out=64
 - norm=None, dropout=0, batch_size=2048, methodology=ours
 - ratio=0.1822305564160316, aggregation=mean, treatment=removal, anomaly_detector=nearest_neighbors, sampling=sage
 - model_type=GCNConv, rate_pairs=10, clusters=50, lr=0.005
Epoch 0 / 100, Loss: 4.5456
Epoch 10 / 100, Loss: 3.7924
Epoch 20 / 100, Loss: 3.0968
Epoch 30 / 100, Loss: 2.5100
Epoch 40 / 100, Loss: 2.0480
Epoch 50 / 100, Loss: 1.6982
Epoch 60 / 100, Loss: 1.4190
Epoch 70 / 100, Loss: 1.1968
Epoch 80 / 100, Loss: 1.0250
Epoch 90 / 100, Loss: 0.8854
 - Test Metrics: Accuracy=0.9599, F1=0.9145, Recall=0.9362, Precision=0.8938
 - Validation Metrics: Accuracy=nan, F1=0.0000, Recall=0.0000, Precision=0.0000
Running experiment with seed=114:
 - K=3, layers=2, hidden=256, out=64
 - norm=None, dropout=0, batch_size=2048, methodology=ours
 - ratio=0.1822305564160316, aggregation=mean, treatment=removal, anomaly_detector=nearest_neighbors, sampling=sage
 -

[I 2025-04-14 22:11:10,444] Trial 0 finished with value: 0.9125326523955319 and parameters: {}. Best is trial 0 with value: 0.9125326523955319.


 - Test Metrics: Accuracy=0.9564, F1=0.9070, Recall=0.9287, Precision=0.8864
 - Validation Metrics: Accuracy=nan, F1=0.0000, Recall=0.0000, Precision=0.0000
Done. Results written to wiki-cs_experimentations\wikics_sar_val_1404220029.csv.
Average F1 over valid seeds: 0.9125 ± 0.0028
Best trial:
  Average F1: 0.9125326523955319
  Best parameters:


# Elliptic Bitcoin

In [ ]:
from data_generating import make_pu_dataset,load_dataset
data=load_dataset("elliptic-bitcoin")
data=make_pu_dataset(data)

# Convert the sum of train_mask to a Python int
labeled = data.train_mask.sum().item()  

# If you need the total number of nodes (rows in data.x):
total = data.x.size(0)  # or data.x.shape[0]
#0.018961
ratio=(0.02*(total-labeled)-0.5/0.35*labeled)/(total-labeled -0.5/0.35*labeled)
print(ratio)

0.008049935058661887


In [ ]:
# Convert the sum of train_mask to a Python int
labeled = data.train_mask.sum().item()  

# If you need the total number of nodes (rows in data.x):
total = data.x.size(0)  # or data.x.shape[0]
#0.018961
ratio=(0.022062*(total-labeled)-0.5/0.35*labeled)/(total-labeled -0.5/0.35*labeled)
print(ratio)

#66.1

0.010137078970813966


In [12]:
from train_NNIF_GNN import run_nnif_gnn_experiment
import optuna
from typing import Dict, Any

def objective(trial: optuna.trial.Trial) -> float:
    """
    Objective function for hyperparameter optimization using Optuna.
    It builds a parameter dictionary, calls the experiment function, and
    returns the average F1 score.
    """
    params: Dict[str, Any] = {
        "dataset_name": "elliptic-bitcoin",      
        "mechanism": "SAR2",
        "train_pct": 0.5,
        "alpha": 0.5,
        "K": 6,#trial.suggest_int("K", 2, 10),
        "layers": 2,#trial.suggest_int("layers", 1, 2),
        "hidden_channels": 64,#trial.suggest_categorical("hidden_channels", [64,128, 256]),
        "out_channels": 64,#trial.suggest_categorical("out_channels", [64,128, 256]),
        "norm": None,
        "dropout": 0,#trial.suggest_float("dropout", 0, 0.4),
        "ratio": 0.010137078970813966,#trial.suggest_float("ratio", 0.01, 0.03),
        "aggregation": 'mean',
        "treatment": "removal",#trial.suggest_categorical("treatment", ["removal", "relabeling"]),
        "model_type": 'GCNConv',#trial.suggest_categorical("model_type", ['GCNConv', 'GATConv']),
        "rate_pairs":10,#trial.suggest_int("rate_pairs",1,15),
        "batch_size": 2048,#trial.suggest_categorical("batch_size", [4096,8192,16384]),
        "lr": 0.005,#trial.suggest_float("lr", 1e-4, 1e-2),
        "anomaly_detector":"nearest_neighbors",#trial.suggest_categorical("anomaly_detector",["nearest_neighbors","unweighted"]),
        "seeds": 1,
        "output_csv": "elliptic_bitcoin_cluster.csv",
        "min":0.6,
        "clusters":50,#trial.suggest_categorical("clusters", [100,500,1000]),          
        "num_epochs":50,#trial.suggest_categorical("num_epochs", [50,100,150]),         
        "sampling":"sage",#trial.suggest_categorical("sampling",["neighbor","cluster"]),   
        "val":False      
        }

    
    # Call the experiment function with these parameters.
    # run_nnif_gnn_experiment returns (avg_f1, std_f1)
    avg_f1, std_f1 = run_nnif_gnn_experiment(params)
    
    # We aim to maximize F1 score.
    return avg_f1

# Create an Optuna study to maximize the F1 score.
study = optuna.create_study(direction="maximize")
study.optimize(objective, n_trials=400)

# Print out the best hyperparameters and corresponding F1 score.
print("Best trial:")
trial = study.best_trial
print("  Average F1:", trial.value)
print("  Best parameters:")
for key, value in trial.params.items():
    print(f"    {key}: {value}")


[I 2025-04-15 17:54:20,375] A new study created in memory with name: no-name-3fbbd5a8-5d67-47c1-9323-62e661673c34


Running experiment with seed=654:
 - K=6, layers=2, hidden=64, out=64
 - norm=None, dropout=0, batch_size=2048, methodology=ours
 - ratio=0.011150786867895364, aggregation=mean, treatment=removal, anomaly_detector=nearest_neighbors, sampling=sage
 - model_type=GCNConv, rate_pairs=10, clusters=50, lr=0.005
Epoch 0 / 50, Loss: 132.0312
Epoch 10 / 50, Loss: 12.5536
Epoch 20 / 50, Loss: 3.7619
Epoch 30 / 50, Loss: 1.6689
Epoch 40 / 50, Loss: 0.8631


[I 2025-04-15 18:13:15,186] Trial 0 finished with value: 0.6594922737306843 and parameters: {}. Best is trial 0 with value: 0.6594922737306843.


 - Test Metrics: Accuracy=0.9470, F1=0.6595, Recall=0.5259, Precision=0.8842
 - Validation Metrics: Accuracy=nan, F1=0.0000, Recall=0.0000, Precision=0.0000
Done. Results written to elliptic-bitcoin_experimentations\elliptic_bitcoin_cluster_1504175420.csv.
Average F1 over valid seeds: 0.6595 ± 0.0000
Running experiment with seed=654:
 - K=6, layers=2, hidden=64, out=64
 - norm=None, dropout=0, batch_size=2048, methodology=ours
 - ratio=0.011150786867895364, aggregation=mean, treatment=removal, anomaly_detector=nearest_neighbors, sampling=sage
 - model_type=GCNConv, rate_pairs=10, clusters=50, lr=0.005


[W 2025-04-15 18:14:11,889] Trial 1 failed with parameters: {} because of the following error: KeyboardInterrupt().
Traceback (most recent call last):
  File "c:\Users\romai\Desktop\gnn\gnn_pu\.conda\Lib\site-packages\optuna\study\_optimize.py", line 197, in _run_trial
    value_or_values = func(trial)
                      ^^^^^^^^^^^
  File "C:\Users\romai\AppData\Local\Temp\ipykernel_21344\1434882542.py", line 42, in objective
    avg_f1, std_f1 = run_nnif_gnn_experiment(params)
                     ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\romai\Desktop\gnn\gnn_pu\NNIF-GNN\train_NNIF_GNN.py", line 562, in run_nnif_gnn_experiment
    train_labels, train_proba, train_losses = train_graph(
                                              ^^^^^^^^^^^^
  File "c:\Users\romai\Desktop\gnn\gnn_pu\NNIF-GNN\train_NNIF_GNN.py", line 297, in train_graph
    neg_mask, pos_mask = nnif_detector.get_reliable(features_np, y_labels)
                         ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^

KeyboardInterrupt: 

In [ ]:
from train_NNIF_GNN import run_nnif_gnn_experiment
ratios=[0.24134569855135768,0.18671026440138042,0.31766718352866136,0.1822305564160316]
names=['cora','citeseer','pubmed','wiki-cs']
for i,name in enumerate(names):
    for mechanism in ['SCAR','SAR']:
        for methodology in ['XGBoost','naive', 'NNIF','two_nnif','spy', 'nnpu','imbnnpu']:
            for model_type in ['MLP','SAGEConv','GCNConv']:
                if methodology in ['naive','two_nnif','spy']:
                    out_channels=2
                else:
                    out_channels=1
                print(methodology)
                params = {
                    "dataset_name": name,
                    "mechanism": mechanism,
                    "train_pct": 0.5,
                    "K": 4,
                    "layers": 2,
                    "hidden_channels": 16,
                    "out_channels": out_channels,
                    "norm": None,
                    "dropout": 0,
                    "ratio": ratios[i],
                    "aggregation": 'mean',
                    "treatment": "removal",
                    "model_type": model_type,
                    "rate_pairs":10,
                    "batch_size": 2048,
                    "lr": 0.005,
                    "anomaly_detector":"nearest_neighbors",
                    "seeds": 5,
                    "output_csv": f"{name}_{mechanism}_{methodology}.csv",
                    "min":0.0,
                    "clusters":50,        
                    "num_epochs":50,         
                    "sampling":"sage",
                    "val":False,
                    "methodology": methodology,
                }
                avg_f1, std_f1 = run_nnif_gnn_experiment(params)
                print('___________________________________________________________________________________')
                print(f"Avg F1 for {name} with {mechanism} and {methodology} and {model_type}: {avg_f1:.4f} ± {std_f1:.4f}")
                print('___________________________________________________________________________________')

In [2]:
from train_NNIF_GNN import run_nnif_gnn_experiment
ratios=[0.24134569855135768,0.18671026440138042,0.31766718352866136,0.1822305564160316]
names=['cora','citeseer','pubmed','wiki-cs']
for i,name in enumerate(names):
    for mechanism in ['SCAR','SAR']:
        for methodology in ['XGBoost','naive', 'NNIF','two_nnif','spy', 'nnpu','imbnnpu']:
            if methodology in ['naive','two_nnif','spy']:
                out_channels=2
            else:
                out_channels=1
            print(methodology)
            params = {
                "dataset_name": name,
                "mechanism": mechanism,
                "train_pct": 0.5,
                "K": 4,
                "layers": 2,
                "hidden_channels": 16,
                "out_channels": out_channels,
                "norm": None,
                "dropout": 0,
                "ratio": ratios[i],
                "aggregation": 'mean',
                "treatment": "removal",
                "model_type": 'GCNConv',
                "rate_pairs":10,
                "batch_size": 2048,
                "lr": 0.005,
                "anomaly_detector":"nearest_neighbors",
                "seeds": 5,
                "output_csv": f"{name}_{mechanism}_{methodology}.csv",
                "min":0.0,
                "clusters":50,        
                "num_epochs":50,         
                "sampling":"sage",
                "val":False,
                "methodology": methodology,
            }
            avg_f1, std_f1 = run_nnif_gnn_experiment(params)
            print('___________________________________________________________________________________')
            print(f"Avg F1 for {name} with {mechanism} and {methodology}: {avg_f1:.4f} ± {std_f1:.4f}")
            print('___________________________________________________________________________________')

XGBoost
Running experiment with seed=654:
 - K=4, layers=2, hidden=16, out=1
 - norm=None, dropout=0, batch_size=2048, methodology=XGBoost
 - ratio=0.24134569855135768, aggregation=mean, treatment=removal, anomaly_detector=nearest_neighbors, sampling=sage
 - model_type=GCNConv, rate_pairs=10, clusters=50, lr=0.005
here 2
 - Test Metrics: Accuracy=0.8282, F1=0.0662, Recall=0.0342, Precision=1.0000
 - Validation Metrics: Accuracy=nan, F1=0.0000, Recall=0.0000, Precision=0.0000
Running experiment with seed=114:
 - K=4, layers=2, hidden=16, out=1
 - norm=None, dropout=0, batch_size=2048, methodology=XGBoost
 - ratio=0.24134569855135768, aggregation=mean, treatment=removal, anomaly_detector=nearest_neighbors, sampling=sage
 - model_type=GCNConv, rate_pairs=10, clusters=50, lr=0.005
here 2
 - Test Metrics: Accuracy=0.8256, F1=0.0384, Recall=0.0196, Precision=1.0000
 - Validation Metrics: Accuracy=nan, F1=0.0000, Recall=0.0000, Precision=0.0000
Running experiment with seed=25:
 - K=4, layers=

KeyboardInterrupt: 

In [ ]:
#Varying train_pct
from train_NNIF_GNN import run_nnif_gnn_experiment
ratios=[0.238436,0.224092]
outs=[256,256]
layers=[2,2]
labeled =[409,1340]  
total = [2708,11701]
Ks=[17,14]
hiddens=[256,256]
bs=[2048,1024]
model_types=["GCNConv","GCNConv"]


names=['cora','wiki-cs']
for i,name in enumerate(names):
    for mechanism in ['SCAR','SAR']:
        for methodology in ['naive', 'NNIF','two_nnif','spy', 'nnpu','imbnnpu','ours']:
            for train_pct in [0.5,0.4,0.3,0.2]:
                for model_type in ['MLP','GATConv','GCNConv']:
                    if methodology =="ours" and model_type in ["GATConv","GCNConv"]:
                        continue
                    batch_size=2048
                    mt=model_type
                    K=1
                    layer=2
                    hidden=16
                    ratio=(ratios[i]*(total[i]-labeled[i])-train_pct/0.5*labeled[i])/(total-labeled[i] -train_pct/0.5*labeled[i])
                    
                    if methodology in ['naive','two_nnif','spy']:
                        out_channels=2
                        
                    elif methodology == "ours":
                        out_channels=outs[i]
                        K=Ks[i]
                        hidden=hiddens[i]
                        layer=layers[i]
                        model_type=model_types[i]
                        batch_size=bs[i]
                    else:
                        out_channels=1
                    print(methodology)
                    params = {
                        "dataset_name": name,
                        "mechanism": mechanism,
                        "train_pct": train_pct,
                        "K": K,
                        "layers": layer,
                        "hidden_channels": hidden,
                        "out_channels": out_channels,
                        "norm": None,
                        "dropout": 0,
                        "ratio": ratio,
                        "aggregation": 'mean',
                        "treatment": "removal",
                        "model_type": model_type,
                        "rate_pairs":10,
                        "batch_size": 2048,
                        "lr": 0.005,
                        "anomaly_detector":"nearest_neighbors",
                        "seeds": 5,
                        "output_csv": f"{name}_{mechanism}_{methodology}.csv",
                        "min":0.0,
                        "clusters":50,        
                        "num_epochs":50,         
                        "sampling":"sage",
                        "val":False,
                        "methodology": methodology,
                    }
                    avg_f1, std_f1 = run_nnif_gnn_experiment(params)
                    print('___________________________________________________________________________________')
                    print(f"Avg F1 for {name} with {mechanism} and {methodology}, {model_type},{train_pct}: {avg_f1:.4f} ± {std_f1:.4f}")
                    print('___________________________________________________________________________________')


In [ ]:
#Varying train_pct
from train_NNIF_GNN import run_nnif_gnn_experiment
ratios=[0.238436,0.224092]
outs=[256,256]
layers=[2,2]
labeled =[409,1340]  
total = [2708,11701]
Ks=[17,14]
hiddens=[256,256]
bs=[2048,1024]
model_types=["GCNConv","GCNConv"]
Ks=[5,10,15,20,25]
ratio_mult=[0.6,0.8,1,1.2,1.4]

names=['cora','wiki-cs']
for i,name in enumerate(names):
    for mechanism in ['SCAR','SAR']:
        for mult in ratio_mult:
            for K in Ks:
                methodology == "ours":
                out_channels=outs[i]
                hidden=hiddens[i]
                layer=layers[i]
                model_type=model_types[i]
                ratio=ratios[i]*mult
                batch_size=bs[i]
                params = {
                        "dataset_name": name,
                        "mechanism": mechanism,
                        "train_pct": train_pct,
                        "K": K,
                        "layers": layer,
                        "hidden_channels": hidden,
                        "out_channels": out_channels,
                        "norm": None,
                        "dropout": 0,
                        "ratio": ratio,
                        "aggregation": 'mean',
                        "treatment": "removal",
                        "model_type": model_type,
                        "rate_pairs":10,
                        "batch_size": batch_size,
                        "lr": 0.005,
                        "anomaly_detector":"nearest_neighbors",
                        "seeds": 5,
                        "output_csv": f"{name}_{mechanism}_{methodology}.csv",
                        "min":0.0,
                        "clusters":50,        
                        "num_epochs":50,         
                        "sampling":"sage",
                        "val":False,
                        "methodology": methodology,
                    }
                avg_f1, std_f1 = run_nnif_gnn_experiment(params)
                print('___________________________________________________________________________________')
                print(f"Avg F1 for {name} with {mechanism} and {methodology}, {model_type},{train_pct}: {avg_f1:.4f} ± {std_f1:.4f}")
                print('___________________________________________________________________________________')


In [ ]:
#Varying prior and pollution ratios
from train_NNIF_GNN import run_nnif_gnn_experiment
ratios=[0.24134569855135768,0.18671026440138042,0.31766718352866136,0.1822305564160316]
out=[]
layers=[]

K=[3,6,9,12,15,18,21]
ratios_mult=[0.8,0.9,1,1.1,1.2]
names=['cora','wiki-cs']
for i,name in enumerate(names):
    for mechanism in ['SCAR','SAR']:
        for methodology in ['NNIF','two_nnif', 'nnpu','imbnnpu','ours']:
            for model_type in ['MLP','SAGEConv','GCNConv']:
                for train_pct in [0.5,0.4,0.3,0.2]:
                    if methodology in ['naive','two_nnif','spy']:
                        out_channels=2
                    elif methodology == "ours":
                        out_channels=out[i]
                    else:
                        out_channels=1
                    print(methodology)
                    params = {
                        "dataset_name": name,
                        "mechanism": mechanism,
                        "train_pct": 0.5,
                        "K": 4,
                        "layers": 2,
                        "hidden_channels": 16,
                        "out_channels": out_channels,
                        "norm": None,
                        "dropout": 0,
                        "ratio": ratios[i],
                        "aggregation": 'mean',
                        "treatment": "removal",
                        "model_type": model_type,
                        "rate_pairs":10,
                        "batch_size": 2048,
                        "lr": 0.005,
                        "anomaly_detector":"nearest_neighbors",
                        "seeds": 5,
                        "output_csv": f"{name}_{mechanism}_{methodology}.csv",
                        "min":0.0,
                        "clusters":50,        
                        "num_epochs":50,         
                        "sampling":"sage",
                        "val":False,
                        "methodology": methodology,
                    }
                    avg_f1, std_f1 = run_nnif_gnn_experiment(params)
                    print('___________________________________________________________________________________')
                    print(f"Avg F1 for {name} with {mechanism} and {methodology} and {model_type}: {avg_f1:.4f} ± {std_f1:.4f}")
                    print('___________________________________________________________________________________')

In [ ]:
#Varying prior and pollution ratios
from train_NNIF_GNN import run_nnif_gnn_experiment
ratios=[0.24134569855135768,0.18671026440138042,0.31766718352866136,0.1822305564160316]
out=[]
layers=[]

methodology="ours"
model_types=['MLP','SAGEConv','GCNConv']
names=['cora','wiki-cs']
for i,name in enumerate(names):
    for mechanism in ['SCAR','SAR']:
        for treatment in ['removal','relabeling']:
            for anomaly_detector in ['nearest_neighbors','unweighted']:
                for model_type in ['MLP','SAGEConv','GCNConv']:
                    print(methodology)
                    params = {
                        "dataset_name": name,
                        "mechanism": mechanism,
                        "train_pct": 0.5,
                        "K": 4,
                        "layers": 2,
                        "hidden_channels": 16,
                        "out_channels": out_channels,
                        "norm": None,
                        "dropout": 0,
                        "ratio": ratios[i],
                        "aggregation": 'mean',
                        "treatment": "removal",
                        "model_type": model_type,
                        "rate_pairs":10,
                        "batch_size": 2048,
                        "lr": 0.005,
                        "anomaly_detector":"nearest_neighbors",
                        "seeds": 5,
                        "output_csv": f"{name}_{mechanism}_{methodology}.csv",
                        "min":0.0,
                        "clusters":50,        
                        "num_epochs":50,         
                        "sampling":"sage",
                        "val":False,
                        "methodology": methodology,
                    }
                    avg_f1, std_f1 = run_nnif_gnn_experiment(params)
                    print('___________________________________________________________________________________')
                    print(f"Avg F1 for {name} with {mechanism} and {methodology} and {model_type}: {avg_f1:.4f} ± {std_f1:.4f}")
                    print('___________________________________________________________________________________')

In [ ]:
#Varying prior and pollution ratios
from train_NNIF_GNN import run_nnif_gnn_experiment
ratios=[0.24134569855135768,0.18671026440138042,0.31766718352866136,0.1822305564160316]
out=[]
layers=[]

methodology="ours"
model_types=['MLP','SAGEConv','GCNConv']
samplings=['sage','cluster','shine']
names=['cora','wiki-cs']
for i,name in enumerate(names):
    for mechanism in ['SCAR','SAR']:
        for sampling in samplings:
            for model_type in ['MLP','SAGEConv','GCNConv']:
                print(methodology)
                params = {
                        "dataset_name": name,
                        "mechanism": mechanism,
                        "train_pct": 0.5,
                        "K": 4,
                        "layers": 2,
                        "hidden_channels": 16,
                        "out_channels": out_channels,
                        "norm": None,
                        "dropout": 0,
                        "ratio": ratios[i],
                        "aggregation": 'mean',
                        "treatment": "removal",
                        "model_type": model_type,
                        "rate_pairs":10,
                        "batch_size": 2048,
                        "lr": 0.005,
                        "anomaly_detector":"nearest_neighbors",
                        "seeds": 5,
                        "output_csv": f"{name}_{mechanism}_{methodology}.csv",
                        "min":0.0,
                        "clusters":50,        
                        "num_epochs":50,         
                        "sampling":"sage",
                        "val":False,
                        "methodology": methodology,
                    }
                avg_f1, std_f1 = run_nnif_gnn_experiment(params)
                print('___________________________________________________________________________________')
                print(f"Avg F1 for {name} with {mechanism} and {methodology} and {model_type}: {avg_f1:.4f} ± {std_f1:.4f}")
                print('___________________________________________________________________________________')

In [ ]:
from train_NNIF_GNN import run_nnif_gnn_experiment
ratios=[0.24134569855135768,0.18671026440138042,0.31766718352866136,0.1822305564160316]
out=[]
layers=[]

methodology="ours"
model_types=['MLP','SAGEConv','GCNConv','GATConv','GINConv']
names=['cora','wiki-cs']
for i,name in enumerate(names):
    for mechanism in ['SCAR','SAR']:
        for model_type in model_types:
            print(methodology)
            params = {
                        "dataset_name": name,
                        "mechanism": mechanism,
                        "train_pct": 0.5,
                        "K": 4,
                        "layers": 2,
                        "hidden_channels": 16,
                        "out_channels": out_channels,
                        "norm": None,
                        "dropout": 0,
                        "ratio": ratios[i],
                        "aggregation": 'mean',
                        "treatment": "removal",
                        "model_type": model_type,
                        "rate_pairs":10,
                        "batch_size": 2048,
                        "lr": 0.005,
                        "anomaly_detector":"nearest_neighbors",
                        "seeds": 5,
                        "output_csv": f"{name}_{mechanism}_{methodology}.csv",
                        "min":0.0,
                        "clusters":50,        
                        "num_epochs":50,         
                        "sampling":"sage",
                        "val":False,
                        "methodology": methodology,
                    }
            avg_f1, std_f1 = run_nnif_gnn_experiment(params)
            print('___________________________________________________________________________________')
            print(f"Avg F1 for {name} with {mechanism} and {methodology} and {model_type}: {avg_f1:.4f} ± {std_f1:.4f}")
            print('___________________________________________________________________________________')